In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import string
import os
from tqdm import tqdm

In [2]:
all_tinh = pd.read_csv('Bo_4T/tinh.csv',index_col='tinh')

In [3]:
def thongke(sentence,tu_khoa,tu_di_kem,tu_cam):
    sentence = str(sentence).translate(str.maketrans('', '', string.punctuation)).lower()
    K = np.zeros(len(tu_khoa))
    cam = False
    ok = False
    for word in tu_cam:
        if word in sentence:
            cam = True
            break
    if cam:
        return K
    for word in tu_di_kem:
        if word in sentence:
            ok = True
            break
    if not ok:
        return K
    for i in range(len(tu_khoa)):
        if tu_khoa[i] in sentence:
            K[i] = 1
    return K


In [4]:
def tim_user(tinh,all_tinh):
    PATH = f"Bo_4T/tinh/{all_tinh.loc[tinh,'file_name']}.csv"
    df_tinh = pd.read_csv(PATH)
    df_tinh['id'] = df_tinh.iloc[:,11].apply(lambda x:x.split('/')[-1])
    data_of_user = []
    users = df_tinh['id'].unique()
    return users, df_tinh

In [5]:
def thongkecactinh(users ,df_tinh, tu_khoa, tu_di_kem, tu_cam):
    data_of_user = []
    for user in tqdm(users):
        df_sample = df_tinh[df_tinh['id'] == user]
        content = df_sample.iloc[:,5]
        thongketinh = []
        for i in range(len(content)):
            K = thongke(content.iloc[i],tu_khoa,tu_di_kem,tu_cam)
            thongketinh.append(K)
        result = np.sum(np.stack(thongketinh,axis = 0),axis = 0)
        data_of_user.append(result)
    data_of_user = np.array(data_of_user)
    return data_of_user

In [6]:
def thongke_n_nhom_for_user(tinh,all_tinh):
    users, df_tinh = tim_user(tinh,all_tinh)
    for i in range(1,6):
        a_words = pd.read_csv(f'Bo_4T/add_word/n{i}.csv')
        tu_di_kem = a_words.keyword.values
        b_words = pd.read_csv(f'Bo_4T/stop_word/n{i}.csv')
        tu_cam = b_words.keyword.values
        keywords = pd.read_csv('Bo_4T/keyword.csv')
        tu_khoa = keywords.keyword.values
        tu_di_kem = list(map(lambda x:x.lower(),tu_di_kem))
        tu_cam = list(map(lambda x:x.lower(),tu_cam))
        tu_khoa = list(map(lambda x:x.lower(),tu_khoa))
        ketqua = thongkecactinh(users, df_tinh, tu_khoa, tu_di_kem, tu_cam)

        if not os.path.exists(f'Bo_4T/result_for_user/{tinh}'):
            os.makedirs(f"Bo_4T/result_for_user/{tinh}")

        df_ketqua = pd.DataFrame(data = ketqua, 
                  index = users, 
                  columns = tu_khoa).astype(int)    

        df_ketqua.to_csv(f'Bo_4T/result_for_user/{tinh}/user_n{i}.csv')

In [7]:
for i in range(len(all_tinh)):
    thongke_n_nhom_for_user(all_tinh.index[i],all_tinh)

  5%|▍         | 1576/33895 [00:15<05:18, 101.41it/s]

KeyboardInterrupt: 